In [27]:
from datasets import load_dataset

disc_med_sft = load_dataset("Flmc/DISC-Med-SFT", cache_dir='./cache')['train']

In [28]:
# peek
disc_med_sft

Dataset({
    features: ['_id', 'source', 'conversation'],
    num_rows: 464898
})

In [31]:
# preprocess
from sklearn.model_selection import train_test_split

inputs = list[str]()
labels = list[str]()
conversation_pair = list[tuple]()
conversations_trunks = disc_med_sft['conversation']
conversations_trunk = list[dict]()
for conversation_trunk in conversations_trunks:
    for a_conversation in conversation_trunk:
        conversations_trunk.append(a_conversation)
conversations = list[dict]()
for index, conversation in enumerate(conversations_trunk):
    if conversation['role'] == 'user':
        conversations.append((conversation, conversations_trunk[index + 1]))
for conversation in conversations:
    inputs.append(conversation[0]['content'])
    labels.append(conversation[1]['content'])
print(f'{len(inputs)} inputs, {len(labels)} labels')
train_inputs, train_labels, eval_inputs, eval_labels = train_test_split(inputs, labels, test_size=0.2, random_state=42)
{
    'train inputs': train_inputs[:10],
    'train labels': train_labels[:10],
    'eval inputs': eval_inputs[:10],
    'eval labels': eval_labels[:10],
}

1385471 inputs, 1385471 labels


{'train inputs': ['谢谢陈医生热心解答，想再次求教您一下。像我这样长期手淫或类前列腺炎症状，有没有可能对未来的子女造成出生缺陷或智力低下？',
  '谢谢医生的解答，我会按照您的建议去做检查的。',
  '宝宝可能是用手揉了眼睛，患眼外眼角眼白红了，有许多红色的血丝，问题大不大，就用托百士药水，贝复舒凝胶，托百士膏，可以解决问题吗。可以解决的话，用法上还是一天3次，晚上多点几次药膏吗，需要调整吗。麻烦您了于主任，期盼您的答复，谢谢。昨天晚上没有看好孩子，宝宝可能是用手揉了眼睛，患眼外眼角眼白红了，有许多红色的血丝，这个怎么办啊。目前用的托百士药水，贝复舒凝胶，托百士膏就这3种药可以解决问题吗，用法需要调整吗。麻烦您了于主任，期盼您的回复，谢谢',
  '谢谢医生的建议。',
  '你好门医生，可以注射骨水泥吗？需要做什么检查？',
  '肺水肿能治好吗，同时患有高血压和糖尿病。忽然喘气困难',
  '凭你的 经验 这初步认为什么毛',
  '脂溢脱发。产后6个多月开始拔罐减肥，饮食单一，1个多月的时间里减了18斤，产后9个多月开始脱发，晚睡，大便干，晚尿最少有1到2次，掉头到现在有2个月时间己掉了一半了。',
  '那如何可以排除！还是怎样确诊？',
  '我是办公室工作，每天长时间坐着。工作压力确实有点大。'],
 'train labels': ['这个...',
  '我没有舌苔照片，但我可以描述一下我的舌苔情况吗？',
  '医生，我听说椎管狭窄会导致双下肢沉重症状，您觉得我这种情况可能是梨状肌综合征吗？我可以携带检查结果来贵院住院治疗吗？',
  '我之前做过超声检查，可以提供给您参考吗？',
  '怀孕初期，各项数值低，想保胎。末次月经2015年11月1日，打了破卵针后，11月21日排卵，12月8日查出怀孕，抽血绒促62.9miu/ml，孕酮11.765ng/ml，雌二醇266pg/ml。医生开了黄体酮，地屈孕酮片和参茸保胎丸，12月10日复查，孕酮涨到27，雌二醇涨到300，但绒促增长很慢，才99。12月14日复查，绒促涨到342，孕酮30，雌二醇降到154。医生说不好，让停药，停药后12月18日复查，绒促只涨到532，孕酮降到5.57，雌二醇降到113。12月18日做B超，显示宫内见稍强回声伴小无回声，直径2mm',
  '牙周炎，

In [25]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_checkpoint = "Qwen/Qwen2.5-0.5B"
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B", cache_dir='./cache')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, cache_dir='./cache')
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [33]:
# prepare train data
from datasets import Dataset


max_len = 68
max_size = 10
tokenized_train_inputs = tokenizer(
    train_inputs[:max_size], 
    padding='max_length',
    max_length=max_len,
    truncation=True, 
    return_tensors="pt"
)
tokenized_train_labels = tokenizer(
    train_labels[:max_size], 
    padding='max_length',
    max_length=max_len,
    truncation=True, 
    return_tensors="pt"
)
tokenized_eval_inputs = tokenizer(
    eval_inputs[:max_size], 
    padding='max_length',
    max_length=max_len,
    truncation=True, 
    return_tensors="pt"
)
tokenized_eval_labels = tokenizer(
    eval_labels[:max_size], 
    padding='max_length',
    max_length=max_len,
    truncation=True, 
    return_tensors="pt"
)
train_dataset_raw = {
    'input_ids': tokenized_train_inputs['input_ids'],
    'attention_mask': tokenized_train_inputs['attention_mask'],
    'labels': tokenized_train_labels['input_ids']
}
eval_dataset_raw = {
    'input_ids': tokenized_eval_inputs['input_ids'],
    'attention_mask': tokenized_eval_inputs['attention_mask'],
    'labels': tokenized_eval_labels['input_ids']
}
train_dataset = Dataset.from_dict(train_dataset_raw)
eval_dataset = Dataset.from_dict(eval_dataset_raw)
{'train_dataset': train_dataset_raw,'eval_dataset': eval_dataset_raw}

{'train_dataset': {'input_ids': tensor([[102570, 100348, 103998, 110576, 106185,   3837,  99172, 103989,  30918,
            99182,  87026, 100158,   1773,  65101,  35946,  99654, 101930,  44934,
           108673,  57191,  21515, 116019, 100439, 101368,   3837, 104710,  87267,
            32664, 105735, 105376, 101090, 102246, 106136,  57191, 107941, 111616,
            11319, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
           151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
           151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
           151643, 151643, 151643, 151643, 151643],
          [102570, 103998,   9370, 106185,   3837, 105351, 101892, 101214, 101898,
           105715, 101071,   9370,   1773, 151643, 151643, 151643, 151643, 151643,
           151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
           151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
     

In [35]:
# train
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=32,
    per_device_train_batch_size=17,
    per_device_eval_batch_size=17,
    gradient_accumulation_steps=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
).train()

Epoch,Training Loss,Validation Loss
1,No log,13.663263
2,No log,13.643941
3,No log,13.625116
4,No log,13.606952
5,No log,13.589076
6,No log,13.571695
7,No log,13.554918
8,No log,13.538576
9,No log,13.522751
10,0.816900,13.507414


In [36]:
# save
lora_model.save_pretrained("./model/qwen2.5-0.5b-disc-med-sft")

In [37]:
# test
model = AutoModelForCausalLM.from_pretrained('./model/qwen2.5-0.5b-disc-med-sft')
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe('足部骨折。你好大夫，谢谢您百忙中的时间。请问骨折对位可以吗？内侧契骨是稍有错位吗？') # 您好，我很高兴能为您提供帮助，根据您的描述，骨折的对位情况还可以。但是，为了更准确地评估情况，我是否可以看一下术前的片子呢？

model.safetensors:  10%|9         | 94.4M/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'generated_text': '足部骨折。你好大夫，谢谢您百忙中的时间。请问骨折对位可以吗？内侧契骨是稍有错位吗？谢谢您！\n根据您的描述，您可能患有足部骨折。骨折对位是指骨折部位与正常位置的关系，如果骨折对位良好，可以恢复到正常位置。如果骨折对位不良，可能需要进行手术治疗。\n\n关于内侧契骨的对位情况，需要根据具体的骨折类型和位置来判断。一般来说，如果骨折对位良好，内侧契骨可能不会出现错位。但是，如果骨折对位不良，可能需要进行手术治疗。\n\n建议您尽快就医，进行详细的检查和诊断，以便制定合适的治疗方案。'}]